### 1. Utilitários

In [1]:
import pandas as pd
import os

In [2]:
# Lista de 3 valores:
# Caminho dos arquivo no diretório. (str)
# Nome para Dataframe. (str)
# Nome para subtituir coluna "Total" de cada arquivo. (str)
dados = [
    (os.path.join("..", "Área de Culturas Permanentes (hectares)", "Área colhida permanente 1999-2020.csv"), "df_permanente", "Culturas P. Colhidas (hec)"),
    (os.path.join("..", "Producao-Milhos", "Produção - milho(R$).xls"), "df_milho", "Producao de Milho (R$)"),
    (os.path.join("..", "Producao-Soja", "Produção - soja (R$).csv"), "df_soja", "Producao de Soja (R$)"),
    (os.path.join("..", "Despesa Orçamentária (R$)", "Despesa por função - gestão ambiental, agricultura e organização agrária - empenhada - estadual.xls"), "df_despesa", "Despesa Orçamentária (R$)"),
    (os.path.join("..", "Focos de Incendio", "firefoci.csv"), "df_incendio", "Focos de Incêndio"),
    (os.path.join("..", "Área de Culturas Temporárias (hectares)", "Área colhida temporária 1999-2020.csv"), "df_temporaria", "Culturas T. Colhidas (hec)"),
    (os.path.join("..", "Área Plantada ou Colhida (hectares)", "Área plantada ou destinada à colheita - total.xls"), "df_plantada_colhida", "Área Plantada ou Colhida (hec)"),
    (os.path.join("..", "PIB", "PIB 1999-2020.csv"), "df_pib", "PIB Estadual"),
    (os.path.join("..", "Valor da Produção Agrícola (R$)", "Valor de Produção 1999-2020.csv"), "df_producao", "PIB Agrícula Estadual") 
]

# Lista dos estados que queremos filtrar
estados = ["Acre", "Amazonas", "Amapá", "Maranhão", "Pará", "Roraima", "Tocantins", "Rondônia", "Mato Grosso"] 

# Lista de anos como strings
anos_str = [str(year) for year in range(1999, 2021)]

### 1. Pré-processamento

O objetivo final é unir os valores anuais de 1999 a 2020 de cada variável estadual dos 9 estados da região em uma tabela única, para futura implementação de um algoritmo de machine learning.

In [3]:
for arquivo, nome_df, coluna in dados:
    try:
        # Ler arquivos .csv com separador ',', se não, ler arquivo .xls
        try:
            df = pd.read_csv(arquivo, sep=",")
        except:
            df = pd.read_excel(arquivo)

        # Normalizar os nomes das colunas (remover espaços extras)
        df.columns = df.columns.str.strip()

        # Configurar Pandas para exibir números inteiros completos
        pd.options.display.float_format = '{:.2f}'.format
        
        # Remover colunas "Código" e "Sigla", se existirem
        colunas_para_remover = [col for col in ["Codigo", "Código", "Sigla"] if col in df.columns]
        if colunas_para_remover:
            df = df.drop(columns=colunas_para_remover)
            print(f"🗑️ Colunas removidas de {nome_df}: {colunas_para_remover}")

        # Filtrar os estados desejados
        if "Estado" in df.columns:
            df = df[df["Estado"].isin(estados)]
            print(f"🌎 Filtrados apenas os estados desejados em {nome_df}")

        # Armazenar na variável correta
        globals()[nome_df] = df
        print(f"✔ {arquivo} carregado como {nome_df}")
        
    except Exception as e:
        print(f"Erro ao ler {arquivo}: {e}")

# Exibir quais DataFrames foram carregados
print("\n📂 DataFrames disponíveis:", list(nome_df for _, nome_df, _ in dados))

🗑️ Colunas removidas de df_permanente: ['Codigo', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_permanente
✔ ..\Área de Culturas Permanentes (hectares)\Área colhida permanente 1999-2020.csv carregado como df_permanente
🗑️ Colunas removidas de df_milho: ['Codigo', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_milho
✔ ..\Producao-Milhos\Produção - milho(R$).xls carregado como df_milho
🗑️ Colunas removidas de df_soja: ['Sigla']
🌎 Filtrados apenas os estados desejados em df_soja
✔ ..\Producao-Soja\Produção - soja (R$).csv carregado como df_soja
🗑️ Colunas removidas de df_despesa: ['Codigo', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_despesa
✔ ..\Despesa Orçamentária (R$)\Despesa por função - gestão ambiental, agricultura e organização agrária - empenhada - estadual.xls carregado como df_despesa
✔ ..\Focos de Incendio\firefoci.csv carregado como df_incendio
🗑️ Colunas removidas de df_temporaria: ['Código', 'Sigla']
🌎 Filtrados apenas os estados desejados em df_

### 2. Transformação formato Wide para Long

Os DataFrames estão no formato longo, com 22 colunas representando os anos de 1999 a 2020, exceto no Data frame "Focos de incêndio", em que as colunas representam os 9 estados.
A função .melt() permite transformar essas colunas em linhas, facilitando a análise e manipulação dos dados.

In [4]:
df_incendio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Ano          22 non-null     int64  
 1   Acre         22 non-null     float64
 2   Amapá        22 non-null     float64
 3   Amazonas     22 non-null     float64
 4   Maranhão     22 non-null     float64
 5   Mato Grosso  22 non-null     float64
 6   Pará         22 non-null     float64
 7   Rondônia     22 non-null     float64
 8   Roraima      22 non-null     float64
 9   Tocantins    22 non-null     float64
dtypes: float64(9), int64(1)
memory usage: 1.8 KB


In [5]:
for arquivo, nome_df, coluna in dados:
    df = globals()[nome_df]
    try:
        df = df.melt(id_vars=['Estado'], value_vars=anos_str, var_name='Ano', value_name= f'{coluna}')
    except:
        #alterações df_incendio
        df["Ano"] = df['Ano'].astype(str)
        df = df.melt(id_vars=["Ano"], value_vars=estados, var_name="Estado", value_name= f'{coluna}')
    globals()[nome_df] = df

In [6]:
df_incendio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198 entries, 0 to 197
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ano                198 non-null    object 
 1   Estado             198 non-null    object 
 2   Focos de Incêndio  198 non-null    float64
dtypes: float64(1), object(2)
memory usage: 4.8+ KB


### 3. Merge do DataFrame final

In [7]:
df_final = df_permanente.merge(df_milho, how='outer', on=["Ano", "Estado"])\
                            .merge(df_soja, how='outer', on=["Ano", "Estado"] )\
                            .merge(df_despesa, how='outer', on=["Ano", "Estado"])\
                            .merge(df_temporaria, how='outer', on=["Ano", "Estado"])\
                            .merge(df_plantada_colhida, how='outer', on=["Ano", "Estado"])\
                            .merge(df_pib, how='outer', on=["Ano", "Estado"])\
                            .merge(df_producao, how='outer', on=["Ano", "Estado"])\
                            .merge(df_incendio, how='outer', on=["Ano", "Estado"])
df_final

,Estado,Ano,Culturas P. Colhidas (hec),Producao de Milho (R$),Producao de Soja (R$),Despesa Orçamentária (R$),Culturas T. Colhidas (hec),Área Plantada ou Colhida (hec),PIB Estadual,PIB Agrícula Estadual,Focos de Incêndio
0,Acre,1999,9864,18904.94,135.89,16480351.00,81922,91787,1866542.32,280900.59,347.00
1,Amapá,1999,1283,663.59,NaN,2869221.75,6903,8235,2128307.03,54019.05,101.00
2,Amazonas,1999,61397,12572.44,875.29,17685970.41,150055,213964,13933920.08,623336.20,1048.00
3,Maranhão,1999,28896,96560.50,243966.36,15239850.98,1130797,1180275,10308256.16,1174349.20,8712.00
4,Mato Grosso,1999,91480,287455.01,3470093.09,43232165.00,4259560,4445242,12364904.96,6462377.66,44307.00
...,...,...,...,...,...,...,...,...,...,...,...
193,Mato Grosso,2020,37589,10090285.97,22939332.03,555308252.18,17149919,17188641,178649564.00,41799931.48,47708.00
194,Pará,2020,651600,346956.98,1428548.90,380705994.45,1310866,1968093,215935604.00,7981858.68,38603.00
195,Rondônia,2020,83545,396740.36,884772.42,210685707.64,721826,813919,51598741.00,2196163.25,11145.00
196,Roraima,2020,10007,47137.95,162262.75,67296396.75,84045,94052,16024276.00,422452.65,1930.00


In [8]:
df_final.to_csv("tabela_final", index = False, encoding="utf-8")
path = os.getcwd()

print("📂 Dataframe salvo em: ", path) 

📂 Dataframe salvo em:  d:\studs\git\LabSesing-case-study\Código
